In [28]:
import yfinance as yf
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from itertools import combinations
import random
import time

In [29]:
# asking user for risk appetite 
desired_risk = input("Enter desired risk (High, Medium, Low): ").lower()
risk_thresholds = {
    "low": (0, 0.8),
    "medium": (0.8, 1.2),
    "high": (1.2, float('inf'))
}
if desired_risk not in risk_thresholds:
    raise ValueError("Invalid risk level. Choose from 'high', 'medium', or 'low'.")

# asking user for stock symbols
stocks_given = input("Enter stock symbols (separated by commas): ").replace(" ", "").split(",")
if len(stocks_given) < 15:
    raise ValueError("Please provide at least 15 stock symbols.")
market_given = "^GSPC"
tickers_list = stocks_given + [market_given]

# downloading data
print("Downloading data...")
stock_data = yf.download(tickers_list, period='5y', interval='1d')['Close'].dropna()
returns = stock_data.pct_change().dropna()
returns.rename(columns={market_given: 'Market'}, inplace=True)

# setting risk free rate of return and expected market return
rf_annual = 0.0435
rf_daily = rf_annual / 252
expected_market_return = returns['Market'].mean() * 252
lower, upper = risk_thresholds[desired_risk]

# asking user for time limit they want to give
stocks_wanted = int(input("How many stocks do you want a portfolio of?: "))
start_time = time.time()
time_limit = int(input("Enter time limit in seconds (The higher the better): "))
print(f"Searching best {stocks_wanted}-stock portfolio for {time_limit} seconds...")

# setting the best for itertools
best_combo = None
best_beta = None
best_return = -np.inf
best_returns_df = None
attempts = 0

# running combinations and finding the best one
for combo in combinations(stocks_given, stocks_wanted):
    if time.time() - start_time > time_limit:
        break

    portfolio_returns = returns[list(combo)].mean(axis=1)
    excess_portfolio = portfolio_returns - rf_daily
    excess_market = returns['Market'] - rf_daily

    X = excess_market.values.reshape(-1, 1)
    y = excess_portfolio.values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    beta = model.coef_[0][0]

    if lower <= beta < upper:
        expected_return = rf_annual + beta * (expected_market_return - rf_annual)
        if expected_return > best_return:
            best_combo = combo
            best_beta = beta
            best_return = expected_return
            best_returns_df = portfolio_returns.to_frame(name="Portfolio")
            best_returns_df['Market'] = returns['Market']

    attempts += 1

# output
if best_combo:
    print("\n🏆 Best Portfolio Found in Time Limit:")
    print(f"Stocks: {best_combo}")
    print(f"Beta: {best_beta:.4f}")
    print(f"Expected Annual Return: {best_return:.4%}")
    print(f"Expected Market Return: {expected_market_return:.4%}")
    print(f"Combinations Tested: {attempts}")

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.scatter(best_returns_df.index, best_returns_df['Portfolio'], label='Portfolio')
    plt.scatter(best_returns_df.index, best_returns_df['Market'], label='Market', alpha=0.7)
    plt.title(f"Best Portfolio (within {time_limit}s) vs Market")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("\n❌ No matching portfolio found in 15 seconds.")
    print(f"Combinations Tested: {attempts}")

KeyboardInterrupt: Interrupted by user